### Инструкция по выполнению
1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.


In [49]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
import pandas as pd

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., 'zwarte', 'zwork', 'zyklon'],
      dtype=object)

In [12]:
X.shape

(1786, 28382)

In [16]:
feature_mapping = X.get_feature_names()
for x in X:
    print(x)

AttributeError: get_feature_names not found

In [17]:
X_train = X
y_train = newsgroups.target

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.svm import SVC

In [26]:
import numpy as np

In [30]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [34]:
best_estimator = gs.best_estimator_
best_estimator.coef_

<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

In [38]:
result_test = pd.DataFrame(best_estimator.coef_.todense())

In [39]:
result_test = result_test.abs()

In [40]:
result_test = result_test.sort_values(0, axis=1)

In [41]:
index = result_test.iloc[:,-10:].columns

In [42]:
names = vectorizer.get_feature_names_out()

In [43]:
answer = []
for i in index:
    answer.append(names[i])
    print(names[i], end=' ')

sci keith bible religion sky moon atheists atheism god space 

In [44]:
answer = np.sort(answer)

In [45]:
for name in answer:
    print(name, end=' ')

atheism atheists bible god keith moon religion sci sky space 

In [46]:
gs.best_params_

{'C': 1.0}

In [48]:
text_file = open("data.txt", "w")
text_file.write('atheism atheists bible god keith moon religion sci sky space')
text_file.close()